# Ciclopi Case Study

**Ciclopi** (www.ciclopi.eu) is a bike sharing service that is operating in Pisa since 2013. It is a station-based service: a registered user may pick up a bike from one of the stations in the town and can drop it in another station.

The company running the service has provided us a dataset with the logs of all the operations on the system during 2015. An __operation__ is a digital trace containing the description of a pick of a bike from a dock in a station, the duration of the renting, and the drop off at the dock of the destination station. All these data are provided as a single table with multiple attributes (in italian):
 * Comune: The municipality where the station is located (in our case is Pisa)
 * Prelievo (Pick up):
     * StazPrelievo: The station where the bike is picked up
     * DataOraPrelievo: The timestamp when the bike was taken
     * ColonninaPrelievo: The dock within the station where the bike was taken
 * Deposito (drop off):
     * StazDeposito: The station where the bike is dropped
     * DataOraDeposito: The timestamp when the bike was dropped
     * ColonninaDeposito: The dock within the station where the bike was dropped
 * IDBadge: anonymized identification of the user taking the bike
 * NumeroBadge: anonymized identification of the user taking the bike
 
 
 ## Analytical Environment - Installation and Setup
 We are going to analyze these data using the Python environment and the Jupyter Notebook for managing the data.
 As a reference, we assume there is the Anaconda environment already installed.
 To be sure to have all the requirements aligned for the developing, we will create an Anaconda Environment dedicated for this task
```
$> conda create -n dvva python=3.6 anaconda -y
```

where //dvva// is a mnemonic name assigned to the new environment.
Then we will activate the new created environment:

```
$> source activate dvva
```

and we will install the required packages:
```
conda install plotly=2.5.1  pandas=0.22.0 xlrd=1.1.0  geopandas=0.3.0 
```

The libraries we are installing are:
 * **plotly**: the library used to create charts
 * **pandas**" a library to manage tabular data
 * **xlrd**: a library to read and parse Excel files
 * **geopandas**: extension of the pandas library to handle geometric and geographic data

We are also installing a library to manage and create color palettes:
```
pip install colorlover
```

The previous step is done once for the working computer.
For successive sessions of work, we will only activate the environment:
```
$> source activate dvva
```


 

## Preprocessing and cleaning

In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 10000)

#elimino i warning
import warnings
warnings.filterwarnings("ignore")

We start by importing the data from a file.

In [10]:
ciclopi=pd.read_excel("data_in/Pisa_Operazioni_2015.xlsx")
ciclopi.head()

,Comune,StazPrelievo,DataOraPrelievo,ColonninaPrelievo,StazDeposito,ColonninaDeposito,DataOraDeposito,IDBadge,NumeroBadge
0,Pisa,Palacongressi,2015-01-01 01:14:18.057,24,Sms Biblioteca,8,2015-01-01 01:18:31.380,159962,2367
1,Pisa,Comune Palazzo Blu,2015-01-01 04:10:20.740,2,Comune Palazzo Blu,2,2015-01-01 04:10:31.647,154539,2518
2,Pisa,Comune Palazzo Blu,2015-01-01 04:10:45.180,3,Sms Biblioteca,11,2015-01-01 04:20:04.047,154539,2518
3,Pisa,Palacongressi,2015-01-01 04:11:44.700,10,Sms Biblioteca,9,2015-01-01 04:19:04.143,141015,1989
4,Pisa,Borgo Stretto,2015-01-01 05:01:10.717,7,Paparelli,16,2015-01-01 05:08:14.623,142149,2003


Let's check the dimension of the dataset by inspecting the number of rows and columns - output is (row,columns)

In [11]:
ciclopi.shape

(173332, 9)

We check if there are problems in the data, like missing values, incongruent values, repetitions, etc. 
Let's start by checking the presence of null values.

In [12]:
ciclopi[ciclopi.isnull().any(axis=1)]

,Comune,StazPrelievo,DataOraPrelievo,ColonninaPrelievo,StazDeposito,ColonninaDeposito,DataOraDeposito,IDBadge,NumeroBadge


All the locations in the table contain a value.


Let's check the data type of each column.

In [13]:
ciclopi.dtypes 

Comune               object
StazPrelievo         object
DataOraPrelievo      object
ColonninaPrelievo     int64
StazDeposito         object
ColonninaDeposito     int64
DataOraDeposito      object
IDBadge               int64
NumeroBadge           int64
dtype: object

From the list of data types of each column, some types requires further investigations:
 * The attributes _IDBadge_ and _NumeroBadge_ seems very similar. Which is their content?
 * The attributes with a timestamp (DataOra) do not have a data type related to temporal values

### Removal of redundant data
Let's start by checking if _IDBadge_ and _NumeroBadge_ are redundant. If they are, one of the two columns may be removed.

In [14]:
print(len(ciclopi.IDBadge.unique())) # Let's count the distinct values of the _IDBadge_ column
print(len(ciclopi.NumeroBadge.unique())) # Let's count the distinct values of the _NumeroBadge_ column
len(ciclopi.groupby(by=['NumeroBadge','IDBadge'])) # Let's count if they always occur in pairs

2266
2266


2266

In [15]:
ciclopi=ciclopi.drop('NumeroBadge',axis=1) #drop 'NumeroBadge'

Since all the operations are perfomed in Pisa, we can drop _Comune_ column.

In [16]:
ciclopi=ciclopi.drop('Comune',axis=1)

### Converting data types
Now let's focus on the time related attributes. Convert strings contained in _DataOraPrelievo_ and _DataOraDeposito_ to datetime type.

In [17]:
ciclopi['DataOraPrelievo']=pd.to_datetime(ciclopi['DataOraPrelievo'])
ciclopi['DataOraDeposito']=pd.to_datetime(ciclopi['DataOraDeposito'])
ciclopi[['DataOraPrelievo','DataOraDeposito']].dtypes #check the type of new columns

DataOraPrelievo    datetime64[ns]
DataOraDeposito    datetime64[ns]
dtype: object

In [20]:
ciclopi[['DataOraPrelievo','DataOraDeposito']].head() # overview of the content of the two new columns

,DataOraPrelievo,DataOraDeposito
0,2015-01-01 01:14:18.057,2015-01-01 01:18:31.380
1,2015-01-01 04:10:20.740,2015-01-01 04:10:31.647
2,2015-01-01 04:10:45.180,2015-01-01 04:20:04.047
3,2015-01-01 04:11:44.700,2015-01-01 04:19:04.143
4,2015-01-01 05:01:10.717,2015-01-01 05:08:14.623


We do not need over detailed precision for this timestamps. We can round dates to 1 second

In [21]:
ciclopi['DataOraPrelievo']=ciclopi['DataOraPrelievo'].dt.round('1s') #round date time to 1 second
ciclopi['DataOraDeposito']=ciclopi['DataOraDeposito'].dt.round('1s')

### Enriching data
For our analysis we are going to check how long the rent of the bike is. Since there is no explicit attribute on this property, we derive a new attribute by computing the temporal duration (in seconds) from the pick up till the drop off. We call the new attribute _Durata_ (duration)


In [22]:
ciclopi['Durata']=(ciclopi['DataOraDeposito']-ciclopi['DataOraPrelievo']).dt.seconds 

Let's check some statistical propertied of the new attribute.

In [24]:
ciclopi.describe()

,ColonninaPrelievo,ColonninaDeposito,IDBadge,Durata
count,173332.000000,173332.000000,173332.000000,173332.000000
mean,10.535020,10.508631,152183.715148,1434.432182
std,7.656888,7.655139,26408.404822,6484.701703
min,1.000000,1.000000,94286.000000,0.000000
25%,4.000000,4.000000,134249.000000,348.000000
50%,9.000000,9.000000,156765.000000,506.000000
75%,15.000000,15.000000,167356.000000,750.000000
max,30.000000,30.000000,201333.000000,86388.000000


In this domain, it is relevant to study the use of the service in different time intervals during the year. In particular we may take into account the difference of use during seasons, weekend, weekday, hour of the day, etc.
Thus we explictly extract date parts attributes from the timestamp we parsed above.

In [25]:
ciclopi['DataPrelievo']=ciclopi['DataOraPrelievo'].dt.date
ciclopi['MesePrelievo']=ciclopi['DataOraPrelievo'].dt.month
ciclopi['SettimanaPrelievo']=ciclopi['DataOraPrelievo'].dt.week
ciclopi['GiornoPrelievo']=ciclopi['DataOraPrelievo'].dt.weekday
ciclopi['GiornoPrelievo_st']=ciclopi['DataOraPrelievo'].dt.weekday_name
ciclopi['OrarioPrelievo']=ciclopi['DataOraPrelievo'].dt.time
ciclopi['OraPrelievo']=ciclopi['DataOraPrelievo'].dt.hour

In [26]:
ciclopi['DataDeposito']=ciclopi['DataOraDeposito'].dt.date
ciclopi['SettimanaDeposito']=ciclopi['DataOraDeposito'].dt.week
ciclopi['GiornoDeposito']=ciclopi['DataOraDeposito'].dt.weekday
ciclopi['GiornoDeposito_st']=ciclopi['DataOraDeposito'].dt.weekday_name
ciclopi['OrarioDeposito']=ciclopi['DataOraDeposito'].dt.time
ciclopi['OraDeposito']=ciclopi['DataOraDeposito'].dt.hour

In [27]:
ciclopi.tail()

,StazPrelievo,DataOraPrelievo,ColonninaPrelievo,StazDeposito,ColonninaDeposito,DataOraDeposito,IDBadge,Durata,DataPrelievo,MesePrelievo,SettimanaPrelievo,GiornoPrelievo,GiornoPrelievo_st,OrarioPrelievo,OraPrelievo,DataDeposito,SettimanaDeposito,GiornoDeposito,GiornoDeposito_st,OrarioDeposito,OraDeposito
173327,Pratale,2015-12-31 16:43:49,13,Vittorio Emanuele,5,2015-12-31 17:01:59,156604,1090,2015-12-31,12,53,3,Thursday,16:43:49,16,2015-12-31,53,3,Thursday,17:01:59,17
173328,Porta a Lucca,2015-12-31 18:24:42,6,Vittorio Emanuele,4,2015-12-31 18:34:57,194690,615,2015-12-31,12,53,3,Thursday,18:24:42,18,2015-12-31,53,3,Thursday,18:34:57,18
173329,Stazione F.S.,2015-12-31 18:42:29,27,Ospedale Cisanello,10,2015-12-31 18:55:41,188285,792,2015-12-31,12,53,3,Thursday,18:42:29,18,2015-12-31,53,3,Thursday,18:55:41,18
173330,Teatro Tribunale,2015-12-31 19:44:21,7,Comune Palazzo Blu,13,2015-12-31 19:47:08,156631,167,2015-12-31,12,53,3,Thursday,19:44:21,19,2015-12-31,53,3,Thursday,19:47:08,19
173331,Pratale,2015-12-31 21:35:41,14,Stazione F.S.,27,2015-12-31 21:49:59,126270,858,2015-12-31,12,53,3,Thursday,21:35:41,21,2015-12-31,53,3,Thursday,21:49:59,21


For each day we infer the season. 

Attention: Winter is starting on December 21st and ends to March 21st. Since it is crossing the new year, we make a numeri tranformation, translating all the days of the year by 11 (i.e. moving 21st to eh new year) and then taking the results modulo 366

In [ ]:
ciclopi['Stagione']=pd.cut(
    (ciclopi['DataOraPrelievo'].dt.dayofyear + 11) % 366, 
    [0, 91, 183, 275, 366],
    labels=['inverno', 'primavera', 'estate', 'autunno']
)

In a similar way, we assign to each time of the day the reference to a list of time slots.

In [28]:
ciclopi['FasciaOraria']=pd.cut(
    ciclopi['OraPrelievo'],
    [0, 7, 9, 12, 15, 18, 21, 24],
    labels=['00-07 ', '07-09','09-12', '12-15', '15-18','18-21','21-24']
)

To assess the intensity of use of the service, we want to estimate the time between a drop and the successive pick up of the same user.

In [29]:
ciclopi=ciclopi.sort_values(by=['IDBadge','DataOraPrelievo']) #sort dataset by id Badge and date time

In [30]:
ciclopi['next_IDBadge']=ciclopi['IDBadge'].shift(-1) #save next row id badge in a new column
ciclopi['next_DataOraPrelievo']=ciclopi['DataOraPrelievo'].shift(-1) #save next row release of the bicycle time in a new column
ciclopi['Sosta']=np.nan #create an empty column named sosta
ciclopi['Sosta'][ciclopi['IDBadge']==ciclopi['next_IDBadge']]=(ciclopi['next_DataOraPrelievo']-ciclopi['DataOraDeposito']).dt.seconds

In [31]:
ciclopi=ciclopi.drop(['next_IDBadge','next_DataOraPrelievo'],axis=1)

In [32]:
ciclopi.head()

,StazPrelievo,DataOraPrelievo,ColonninaPrelievo,StazDeposito,ColonninaDeposito,DataOraDeposito,IDBadge,Durata,DataPrelievo,MesePrelievo,SettimanaPrelievo,GiornoPrelievo,GiornoPrelievo_st,OrarioPrelievo,OraPrelievo,DataDeposito,SettimanaDeposito,GiornoDeposito,GiornoDeposito_st,OrarioDeposito,OraDeposito,FasciaOraria,Sosta
94781,Vittorio Emanuele,2015-07-01 13:24:57,5,Sms Biblioteca,2,2015-07-01 13:31:57,94286,420,2015-07-01,7,27,2,Wednesday,13:24:57,13,2015-07-01,27,2,Wednesday,13:31:57,13,12-15,60555.0
96668,Sms Biblioteca,2015-07-06 06:21:12,7,Stazione F.S.,8,2015-07-06 06:31:00,94286,588,2015-07-06,7,28,0,Monday,06:21:12,6,2015-07-06,28,0,Monday,06:31:00,6,00-07,NaN
3205,Borgo Stretto,2015-01-13 18:57:16,9,Stazione F.S.,5,2015-01-13 19:05:34,94310,498,2015-01-13,1,3,1,Tuesday,18:57:16,18,2015-01-13,3,1,Tuesday,19:05:34,19,15-18,55492.0
3511,Stazione F.S.,2015-01-14 10:30:26,25,Stazione F.S.,27,2015-01-14 10:31:36,94310,70,2015-01-14,1,3,2,Wednesday,10:30:26,10,2015-01-14,3,2,Wednesday,10:31:36,10,09-12,17.0
3512,Stazione F.S.,2015-01-14 10:31:53,21,Borgo Stretto,1,2015-01-14 10:39:11,94310,438,2015-01-14,1,3,2,Wednesday,10:31:53,10,2015-01-14,3,2,Wednesday,10:39:11,10,09-12,24785.0


Save and check column names

In [33]:
columns=ciclopi.columns.tolist() 
columns

['StazPrelievo',
 'DataOraPrelievo',
 'ColonninaPrelievo',
 'StazDeposito',
 'ColonninaDeposito',
 'DataOraDeposito',
 'IDBadge',
 'Durata',
 'DataPrelievo',
 'MesePrelievo',
 'SettimanaPrelievo',
 'GiornoPrelievo',
 'GiornoPrelievo_st',
 'OrarioPrelievo',
 'OraPrelievo',
 'DataDeposito',
 'SettimanaDeposito',
 'GiornoDeposito',
 'GiornoDeposito_st',
 'OrarioDeposito',
 'OraDeposito',
 'FasciaOraria',
 'Sosta']

Save the dataframe ciclopi in a new csv file for next notebook

In [34]:
ciclopi.to_csv('data_in/02_ciclopi.csv',sep=';', index=False) #export data into a new csv for next notebook